In [1]:
from docx import Document
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

/home/atharva/anaconda3/envs/pytorchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# def read_word_file(file_path):
#     doc = Document(file_path)
#     full_text = []
#     for para in doc.paragraphs:
#         full_text.append(para.text)
#     return '\n'.join(full_text)

# def read_text_file(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         text = file.read()
#     return text

# document_path = '/home/atharva/Atharva/webapp/documentRetrieval/data/resetPassword.txt'
# document_text = read_text_file(document_path)

In [3]:
document_text = """Title: How to Reset Your Password

If you have forgotten your password, you can reset it by following these steps:

Go to the login page.
Click on the "Forgot Password" link.
Enter your registered email address.
Check your email for a password reset link.
Click the link in the email and follow the instructions to create a new password.
If you do not receive the password reset email within a few minutes, please check your spam folder. If the email is not there, you can request a new password reset email by repeating the steps above.

For further assistance, contact our support team at support@example.com."""

lines = [line.strip() for line in document_text.split('\n') if line.strip()]



In [4]:
# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the document
embeddings = model.encode([lines])

/home/atharva/anaconda3/envs/pytorchEnv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Initialize FAISS index
d = embeddings.shape[1]  # dimension of embeddings
index = faiss.IndexFlatL2(d)

# Add embeddings to the index
index.add(np.array(embeddings, dtype=np.float32))
document_mapping = {i: line for i, line in enumerate(lines)}
# Save the index to a file (optional)
faiss.write_index(index, 'document_embeddings.index')

In [7]:
query = "email"
query_embedding = model.encode([query])

# Search the FAISS index
D, I = index.search(np.array(query_embedding, dtype=np.float32), k=1)  # k is the number of nearest neighbors to retrieve
nearest_neighbor_id = I[0][0]
nearest_neighbor_text = document_mapping[nearest_neighbor_id]
# Get the result
print(f"Nearest neighbor ID: {nearest_neighbor_text}, Distance: {D[0][0]}")

Nearest neighbor ID: Title: How to Reset Your Password, Distance: 1.6131680011749268
